In [1]:
import sys
sys.path.extend([ '../', '../../'])
import torch
import dataloader as dl
from args import Args
from model import Content_Agg, Neigh_Agg, EdgePredictor, Het_classify
from train import train_smote

# Set device to GPU if available, else use CPU
args = Args()
args.mp_imdb()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
data = torch.load('data.pt')
print(data)

HeteroData(
  m={ num_nodes=4666 },
  mp1={ x=[4666, 384] },
  mp2={ x=[4666, 384] },
  mp3={ x=[4666, 384] },
  mp={
    edge_index=[2, 50253],
    edge_weight=[50253],
    y=[4666]
  }
)


In [3]:
print(data['mp'].edge_index)
device = args.device

# Send all x tensors to the device
data['mp1']['x'] = data['mp1']['x'].to(device)
data['mp2']['x'] = data['mp2']['x'].to(device)
data['mp3']['x'] = data['mp3']['x'].to(device)

# Send all y tensors to the device
data['mp']['y'] = data['mp']['y'].to(device)

# Send all edge_index tensors to the device
data['mp']['edge_index'] = data['mp']['edge_index'].to(device)
data['mp']['edge_weight'] = data['mp']['edge_weight'].to(device)

tensor([[   0,    0,    0,  ..., 4616, 4616, 4619],
        [  25,  279,  282,  ..., 4619, 4624, 4624]])


In [4]:
c_train_num = dl.train_num(data['mp'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['mp'].y, c_train_num, args.seed[1], args)
print("train_idx: ", train_idx, len(train_idx))
print("val_idx: ", val_idx, len(val_idx))
print("test_idx: ", test_idx, len(test_idx))

[40, 60, 100] 200
0 543
1 924
2 1473
train_idx:  [2745, 1616, 1260, 3151, 1623, 3752, 1211, 991, 3326, 1341, 2098, 2911, 864, 1976, 2092, 911, 3144, 3024, 2899, 2443, 2621, 3905, 1525, 3056, 3411, 1128, 1915, 4342, 3010, 1233, 691, 389, 3457, 3981, 4367, 729, 1430, 202, 2629, 3077, 3623, 4208, 2009, 1448, 572, 3642, 3944, 952, 3990, 6, 1694, 4396, 4391, 833, 1512, 4183, 1609, 2804, 635, 1371, 1483, 2487, 762, 2436, 2706, 576, 4446, 3647, 646, 1533, 1816, 4366, 2124, 3415, 4448, 1232, 2525, 2063, 4198, 2321, 4092, 2091, 2093, 3084, 2850, 3839, 3297, 3096, 1870, 3333, 406, 3021, 797, 3554, 3614, 4, 3048, 2517, 1950, 4091, 2467, 564, 1642, 751, 3356, 3394, 1996, 2151, 190, 1499, 2507, 1645, 2594, 731, 556, 2662, 1582, 3874, 1099, 1413, 879, 451, 1486, 2585, 694, 2883, 1245, 2956, 1300, 165, 1076, 2674, 2383, 852, 2235, 275, 502, 4021, 2832, 384, 2081, 2178, 2367, 4553, 3250, 2267, 1011, 2120, 4118, 496, 531, 3668, 2644, 664, 3631, 3351, 1557, 1311, 2932, 2282, 4346, 1095, 4068, 3853, 1322

In [5]:
encoder1 = Content_Agg(args.embed_dim, args.dropout)
encoder2 = Neigh_Agg(args.embed_dim, args.dropout)
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder = EdgePredictor(args.embed_dim)
# decoder = torch.load('pretrained/decoder.pth')

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
decoder.to(device)

EdgePredictor(
  (lin): Linear(in_features=256, out_features=256, bias=True)
)

## Training Part

In [6]:
torch.cuda.empty_cache()
train_smote(data, encoder1, encoder2, classifier, decoder, train_idx, val_idx, test_idx, args, train_mode = 'recon')

Epoch [1/200], Loss: 0.1188, Accuracy: 0.3033, Edge Accuracy: 1.0
class 0: 0.1, Class 0: AUC-ROC- 0.3926, F1 Score- 0.5000; class 1: 0.1, Class 1: AUC-ROC- 0.5115, F1 Score- 0.5000; class 2: 0.1, Class 2: AUC-ROC- 0.4607, F1 Score- 0.5000; Macro-Average AUC-ROC: 0.4549; Macro-Average F1 Score: 0.5000
Validation Loss: 0.1186, Validation Accuracy: 0.3310, Validation Edge Accuracy: 1.0
class 0: 0.1, Class 0: AUC-ROC- 0.4220, F1 Score- 0.5000; class 1: 0.1, Class 1: AUC-ROC- 0.5102, F1 Score- 0.5000; class 2: 0.1, Class 2: AUC-ROC- 0.3935, F1 Score- 0.5000; Macro-Average AUC-ROC: 0.4419; Macro-Average F1 Score: 0.5000
Test Loss: 0.1186, Test Accuracy: 0.3033, Test Edge Accuracy: 1.0
class 0: 0.1, Class 0: AUC-ROC- 0.3953, F1 Score- 0.5000; class 1: 0.1, Class 1: AUC-ROC- 0.4741, F1 Score- 0.5000; class 2: 0.1, Class 2: AUC-ROC- 0.4098, F1 Score- 0.5000; Macro-Average AUC-ROC: 0.4264; Macro-Average F1 Score: 0.5000

Epoch [2/200], Loss: 0.1186, Accuracy: 0.3033, Edge Accuracy: 1.0
class 0: 

([0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.30333333333333334,
  0.3011111111111111,
  0.30444444444444446,
  0.30333333333333334,
  0.30444444444444446,
  0.3055555555555556,
  0.3055555555555556],
 [0.4263851851851852,
  0.425425,
  0.42469629629629635,
  0.4239537037037038,
  0.423812037037037,
  0.42643518518518525,
  0.4595777777777778,
  0.4575212962962963,
  0.45748518518518516,
  0.45535648148148145,
  0.44960000000000006,
  0.44386574074074076,
  0.43920740740740744,
  0.43496574074074074,
  0.4298796296296296,
  0.42903981481481485,
  0.42987314814814814,
  0.4392935185185185,
  0.44966018518518514,
  0.4575324074074074,
  0.4637842592592592],
 [0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,
  0.5,


In [7]:
# torch.save(decoder, 'pretrained/decoder.pth')
# torch.save(encoder1, 'pretrained/encoder1.pth')
# torch.save(encoder2, 'pretrained/encoder2.pth')